在这个notebook中，我们

- 已知：hf&hhf模型预测结果，当前仓位
- 目标：输出仓位变动，报单价格

这里的代码做为返回策略线程的基础。

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time

pd.options.mode.chained_assignment = None

In [2]:
hf_portion = 2/3  # hf交易占全部交易的权重
leverage = 1.99  # 杠杆率

capital = 2e8
K = capital * leverage  # 可操作资金
current_pos = np.random.rand(500) # 当前仓位

In [3]:
# 每个tick拿到数据计算因子 --> 放入模型预测 --> 返回策略输出

newdata = np.random.rand(500, 48) # 新拿到的数据
alphas = np.random.rand(500, 23)  # 计算出的因子

# hf（预测10ticks）部分

In [4]:
hf_holding = 10  # 持有周期（单位：tick）
hf_num_newpos = 15  # 每轮分别新做多和做空股数
# hf_weights_d = np.array([1.35, 1.3, 1.25, 1.2, 1.15, 1.1, 1.05, 1, 
#                          0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65])
hf_weights_d = np.array([1.7, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 1, 
                         0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3])
hf_weights_a = np.flip(hf_weights_d)
# keep track of newly initiatied positions from the previous 10 ticks
hf_track_newpos = np.zeros([hf_holding, 500])

In [5]:
hf_loaded_model = pickle.load(open('Models/GB_23_full.sav', 'rb')) # hf 模型

In [6]:
hf_pred = hf_loaded_model.predict(alphas) # shape (500)

In [7]:
hf_longstock = hf_pred.argsort()[-hf_num_newpos:] # 做多哪些股票
hf_shortstock = hf_pred.argsort()[:hf_num_newpos] # 做空哪些股票

hf_newpos = np.zeros(500)

hf_afford = K*hf_portion/(2*hf_num_newpos)/hf_holding/newdata[:,5]

hf_newpos[hf_longstock] = hf_afford[hf_longstock] * hf_weights_a # 非等权做多
hf_newpos[hf_shortstock] = - hf_afford[hf_shortstock] * hf_weights_d # 非等权做空

hf_track_newpos = np.append(hf_track_newpos, np.array([hf_newpos]), axis = 0)[1:]

hf_obj_pos = hf_track_newpos.sum(axis = 0)

# hhf（预测5ticks）部分

In [8]:
hhf_holding = 5  # 持有周期（单位：tick）
hhf_num_newpos = 15  # 每轮分别新做多和做空股数
# hf_weights_d = np.array([1.35, 1.3, 1.25, 1.2, 1.15, 1.1, 1.05, 1, 
#                          0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65])
hhf_weights_d = np.array([1.7, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 1, 
                         0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3])
hhf_weights_a = np.flip(hf_weights_d)
# keep track of newly initiatied positions from the previous 5 ticks
hhf_track_newpos = np.zeros([hhf_holding, 500])

In [9]:
hhf_loaded_model = pickle.load(open('Models/GB_23_full.sav', 'rb')) # hhf 模型

In [10]:
hhf_pred = hhf_loaded_model.predict(alphas) # shape (500)

In [11]:
hhf_longstock = hhf_pred.argsort()[-hhf_num_newpos:] # 做多哪些股票
hhf_shortstock = hhf_pred.argsort()[:hhf_num_newpos] # 做空哪些股票

hhf_newpos = np.zeros(500)

hhf_afford = K*(1-hf_portion)/(2*hhf_num_newpos)/hhf_holding/newdata[:,5]

hhf_newpos[hhf_longstock] = hhf_afford[hhf_longstock] * hhf_weights_a # 非等权做多
hhf_newpos[hhf_shortstock] = -hhf_afford[hhf_shortstock] * hhf_weights_d # 非等权做空

hhf_track_newpos = np.append(hhf_track_newpos, np.array([hhf_newpos]), axis = 0)[1:]

hhf_obj_pos = hhf_track_newpos.sum(axis = 0)

# submission

In [14]:
submit_pos = hf_obj_pos + hhf_obj_pos - current_pos  # 提交仓位变化

# 做多报当前tick卖盘ask5，做空报当前tick买盘bid5（几乎保证成交）
bid5 = newdata[:,16]
ask5 = newdata[:,36]
submit_price = ask5 * (submit_pos > 0) + bid5 * (submit_pos < 0)  # 提交报单价格

若accruacy为$0.53$，优势为$0.6=2\times15/500$。